In [1]:
from emrgpt.model import TimelineBasedEmrGPT
from emrgpt.trainer import TimelineDS
import torch
import dotenv
import pandas as pd

dotenv.load_dotenv('.env', override=True)

*** EMR GPT ***


True

In [4]:
ds = TimelineDS(block_size=24)
ds

In [5]:
model = TimelineBasedEmrGPT(
    n_event_types=len(ds.features),
    d_model=64,
    block_size=24,
    max_len=24,
    n_head=6,
    n_layer=10,
    dropout=0.2,
).to('cuda')

model.load_state_dict(torch.load('cache/savedmodels/TimelineBasedEmrGPT.pt'))
model.eval()
model

TimelineBasedEmrGPT(
  (proj): Linear(in_features=35, out_features=64, bias=True)
  (positional_encoding): FixedPositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (blocks): Sequential(
    (0): AkDecoderBlock(
      (self_attention): AkMultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AkSelfAttentionHead(
            (key): Linear(in_features=64, out_features=10, bias=False)
            (query): Linear(in_features=64, out_features=10, bias=False)
            (value): Linear(in_features=64, out_features=10, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=60, out_features=64, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (feedforward): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=64, out_features=256, bias=True)
          (1): ReLU()
          (2): Linear(in_features=256, out_features=64, bias=True)
          (3): Dropo

In [10]:
g = model.generate(
    seed=torch.tensor([0.0] * len(ds.features), device='cuda').unsqueeze(0).unsqueeze(0)
)
g.shape
g_denorm = ds.denormalize(g.squeeze(0))
df = pd.DataFrame(data=g_denorm.detach().cpu(), columns=ds.features)
df

,heart_rate,sbp,dbp,resp_rate,temperature,spo2,glucose,norepi_eq_rate,vent_hfnc,vent_suppo2,...,globulin,total_protein,aniongap,bicarbonate,bun,calcium,chloride,creatinine,sodium,potassium
0,92.081932,109.032204,57.397167,20.310043,36.841934,95.936295,161.014938,0.000002,0.000075,0.012857,...,2.549053,5.761964,16.153061,21.415558,33.56123,8.016164,104.625427,0.111658,135.776031,4.156446


In [11]:
df[['glucose']]

,glucose
0,161.014938
